<a href="https://colab.research.google.com/github/cjfghk5697/anomaly-detection-competition/blob/main/Add_EfficientNet%20V2%2C%20Parapmeter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기록

## 07-15
다양한 parameter 실천결과
2e-4가 61%로 지금까지 가장 높음. 그에 반해 2e-2는 좋은 결과가 안나옴. 또한 TTA가 큰 역할을 하는 걸로 보이지는 않는다. 그래서 적절히 섞어가며 테스트하면 더욱 좋은 결과를 낼수 있을 것이다. 

예시로 TTA를 했는데 63퍼이고 안하면 66%인 경우가 있다. TTA에 대한 연구와 parameter을 더욱 조사해보는게 맞는 거 같다.

## EfficientNet V2 추가
최근 EfficientNet V2 관련 논문을 읽었는데 정말 인상적이다. 속도와 메모리 모두 줄였는데 efficientnetV1 만큼 괜찮은 성능과 이미지를 progessive resizieng과 mix&mach, curriculum learning을 보고 반해버렸다.. torch vision에서 efficientnet v2를 제공하기에 추가했다.


```python
from torchvision.models import efficientnet_v2_m

self.model = efficientnet_v2_m(pretrained=True)
````

 efficientNet b3와 비슷한 속도를 초반에 지녔지만 후반에 image가 커지기에 아마 느려질 거 같다.

https://dacon.io/competitions/official/235894/overview/description

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd "/content/drive/MyDrive/input/"
#!unzip -q "/content/drive/MyDrive/input/train.zip" 

/content/drive/MyDrive/input


In [3]:
# autoaugment 사용
!git clone https://github.com/DeepVoltaire/AutoAugment.git
!cp /content/drive/MyDrive/input/AutoAugment/autoaugment.py /content/drive/MyDrive/input
!cp /content/drive/MyDrive/input/AutoAugment/ops.py /content/drive/MyDrive/input

fatal: destination path 'AutoAugment' already exists and is not an empty directory.


In [4]:
#!pip3 install timm

In [5]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
#import timm
import random
from typing import Tuple, Sequence, Callable

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time
from PIL import Image
import cv2
from torch.autograd import Variable
from autoaugment import ImageNetPolicy, CIFAR10Policy, SVHNPolicy, SubPolicy

from torchvision.models import efficientnet_v2_s

device = torch.device('cuda')

In [6]:
train_png = sorted(glob('./train/*.png'))

In [7]:
train_y = pd.read_csv('./train_df.csv')

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [8]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (512, 512))
    return img

In [9]:
train_imgs = [img_load(m) for m in tqdm(train_png)]

100%|██████████| 4277/4277 [02:43<00:00, 26.15it/s]


In [10]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    ImageNetPolicy(),

    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])


In [11]:
class Custom_dataset(Dataset):
    def __init__(self, 
                 img_paths, 
                 labels, 
                 mode='train',
                 transforms= Sequence[Callable]
            ) -> None:
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
        self.transforms = transforms

    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy()
            elif augmentation==2:
                img = img[:,::-1].copy()
        if self.mode=='test':
            pass
        img = Image.fromarray(img) # NumPy array to PIL image
        if self.transforms is not None:
            img = self.transforms(img)        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = efficientnet_v2_s(pretrained=True)
        self.dropout=nn.Dropout(p=0.2)
        self.SiLU=nn.SiLU(inplace=False)
        self.classifier = nn.Linear(1000, 88)
        nn.init.xavier_normal_(self.classifier.weight)

    def forward(self, x):
        x = self.model(x)
        x = self.dropout(x)
        x = self.SiLU(x) 
        x = self.classifier(x)

        return x

In [12]:
batch_size = 32
epochs = 50

# Train
trainset  = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train', transforms=transforms_train)
lengths = [int(len(trainset)*0.8), int(len(trainset)*0.2)]


In [13]:
lengths=[lengths[0],lengths[1]+1]

In [14]:
train_dataset, valid_dataset = torch.utils.data.random_split(trainset, lengths)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_dataset, shuffle=True, batch_size=batch_size)

In [15]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

### 모델 학습

In [ ]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

model = Network().to(device)
#optimizer = torch.optim.RAdam(model.parameters(), lr=1e-3)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay = 2e-2)
criterion = nn.CrossEntropyLoss()
use_amp = True
scaler = torch.cuda.amp.GradScaler(enabled=use_amp) 
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=0.001, last_epoch=-1)


valid_loss_list=[]
train_loss_list=[]

for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]

    valid_loss = 0
    valid_pred=[]
    valid_y=[]

    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)

        x, targets_a, targets_b, lam = mixup_data(x, y)
        x, targets_a, targets_b = map(Variable, (x, targets_a, targets_b))

        #outputs = model(x)

        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = mixup_criterion(criterion, pred, targets_a, targets_b, lam)


        #loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
    model.eval()
    with torch.no_grad():
      for i in (valid_loader):
          images = torch.tensor(batch[0], dtype=torch.float32, device=device)
          targets = torch.tensor(batch[1], dtype=torch.long, device=device)



          outputs = model(images)
          loss = criterion(outputs, targets)

          valid_loss += loss.item()/len(valid_loader)
          valid_pred += outputs.argmax(1).detach().cpu().numpy().tolist()
          valid_y += targets.detach().cpu().numpy().tolist()
    
    train_f1 = score_function(train_y, train_pred)
    valid_f1 = score_function(valid_y, valid_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'valid    loss : {valid_loss:.5f}    f1 : {valid_f1:.5f}')
    lr_scheduler.step()

    torch.save(model.state_dict(), f'{train_loss}_{train_f1}.pth')

Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth


  0%|          | 0.00/82.7M [00:00<?, ?B/s]

epoch : 1/50    time : 144s/7073s
TRAIN    loss : 2.12752    f1 : 0.07955
valid    loss : 0.83865    f1 : 0.67018
epoch : 2/50    time : 142s/6803s
TRAIN    loss : 1.62028    f1 : 0.09955
valid    loss : 0.59665    f1 : 0.65873
epoch : 3/50    time : 144s/6761s
TRAIN    loss : 1.49506    f1 : 0.11738
valid    loss : 0.37705    f1 : 0.86667
epoch : 4/50    time : 142s/6536s
TRAIN    loss : 1.38360    f1 : 0.14180
valid    loss : 0.41311    f1 : 0.81513
epoch : 5/50    time : 142s/6381s
TRAIN    loss : 1.40660    f1 : 0.16224
valid    loss : 0.36769    f1 : 0.85417
epoch : 6/50    time : 142s/6251s
TRAIN    loss : 1.34082    f1 : 0.18123
valid    loss : 0.46693    f1 : 0.81978
epoch : 7/50    time : 142s/6099s
TRAIN    loss : 1.34543    f1 : 0.14077
valid    loss : 0.43244    f1 : 0.81176
epoch : 8/50    time : 142s/5967s
TRAIN    loss : 1.29636    f1 : 0.16679
valid    loss : 0.34174    f1 : 0.76190
